<img src="https://raw.githubusercontent.com/patel-zeel/Adhoc/master/ADAIN.PNG" alt="Drawing" style="width: 600px;"/>

In [1]:
# !pip install -qq numpy==1.18.5
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, LSTM, Concatenate, Lambda, Multiply, Reshape, Input, Dropout
from keras.models import Sequential
from keras.initializers import RandomUniform
import keras.backend as K
from tensorflow.random import set_seed
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output
from time import time
import os
import matplotlib.pyplot as plt
from matplotlib import rc

from kgp.layers import GP
from kgp.models import Model
from kgp.losses import gen_gp_loss

rc('font', size=16)
# import logging
# tf.get_logger().setLevel(logging.ERROR)
np.__version__

ModuleNotFoundError: No module named 'tensorflow'

### Defining the model

In [2]:
def ADAIN(time_window, met_dim):
    """
    # Vocab
    met         - meteorology
    dis         - distance
    aq          - air quality
    n_stn       - n_stations
    dim         - dimension
    """
    with tf.device('gpu:1'):
        # Hyperparameters
        do = 0.5 # Drop out
        np.random.seed(0)
        set_seed(0)

        # Defining inputs
        train_dist = Input(shape=(2,None,))
        train_met_aq = Input(shape=(24,met_dim+1,None,)) # +1 for aq
        test_met = Input(shape=(24,met_dim,))

        n_stn = 2 if train_dist.shape[2] is None else train_dist.shape[2]

        # Local station                                                        ## Input                --- Output
        lcl_lstm = LSTM(300, dropout=do)(test_met)                             # (-1,time_window,met)  --- (-1,300)
        lcl_dns = Dense(200, activation='relu')(lcl_lstm)                      # (-1,300)              --- (-1,200) 
        lcl_dns = Dropout(do)(lcl_dns)

        # Defininig shared layers (shared among train stations)
        s_dns1 = Dense(100, activation='relu')
        s_lstm = LSTM(300, activation='relu', dropout=do)
        s_dns2 = Dense(200, activation='relu')

        a_dns1 = Dense(200, activation='relu')
        a_dns2 = Dense(200)
        a_dns3 = Dense(1)

        # Training stations
        att_list = [] # Saving station attention weights
        stn_list = [] # Saving station features

        # Iterating over train stations using shared layers
        for s_i in range(n_stn):                                               ## Input               --- Output
            train_dist_slice = Lambda(lambda x:x[:,:,s_i])(train_dist)         # (-1,dis,n_stn)       --- (-1,dis)
            stn_dns1 = s_dns1(train_dist_slice)                                # (-1,dis)             --- (-1,100)
            stn_dns1 = Dropout(do)(stn_dns1)
            train_met_aq_slice = Lambda(lambda x:x[:,:,:,s_i])(train_met_aq)
            stn_lstm = s_lstm(train_met_aq_slice)                              # (-1,time_window,met) --- (-1,300)
            stn_cat1 = Concatenate()([stn_dns1, stn_lstm])                     # (-1,100)+(-1,300)    --- (-1,400)
            stn_dns2 = s_dns2(stn_cat1)                                        # (-1,400)             --- (-1,200)
            stn_dns2 = Dropout(do)(stn_dns2)
            stn_list.append(stn_dns2)
            ### Attention Mechanism
            att_cat = Concatenate()([stn_dns2, lcl_dns])                       # (-1,200)*2           --- (-1,400)
            att_dns1 = a_dns1(att_cat)                                         # (-1,400)             --- (-1,200)
            att_dns1 = Dropout(do)(att_dns1)
            att_dns2 = a_dns2(att_dns1)                                        # (-1,200)             --- (-1,200)
            att_dns3 = a_dns3(att_dns2)                                        # (-1,200)             --- (-1,1)
            att_list.append(att_dns3)

        ### Normalize Attention
        att_cat1 = Concatenate()(att_list)                                     # (-1,1)*n_stn         --- (-1,n_stn)
        att_cat2 = Lambda(lambda inp: inp/K.sum(inp, axis=0))(att_cat1)        # (-1,n_stn)           --- (-1,n_stn)

        ### Multiply Attention with station features
        stn_cat2 = Concatenate()(stn_list)                                     # (-1,200)*n_station   --- (-1,200*n_stn) 
        stn_cat2 = Lambda(lambda x: K.reshape(x, (-1,200,n_stn)))(stn_cat2)    # (-1,200*n_stn)       --- (-1,200,n_stn)      
        att_cat2 = Lambda(lambda x: K.reshape(x, (-1,1,n_stn)))(att_cat2)      # (-1,n_stn)           --- (-1,1,n_stn)
    #     print(stn_cat2)
        stn_mul = Multiply()([stn_cat2, att_cat2])                             # (-1,200,n_stn)*(-1,1,n_stn)  --- (-1,200,n_stn)
        stn_add = Lambda(lambda x: K.sum(x, axis=2))(stn_mul)                  # (-1,200,n_stn)       --- (-1,200)

        ### Concatenate local and station features
        final_cat = Concatenate()([stn_add, lcl_dns])                          # (-1,200)*2           --- (-1,400)
        final_dns1 = Dense(200, activation='relu')(final_cat)                  # (-1,400)             --- (-1,200)
        final_dns1 = Dropout(do)(final_dns1)
        final_dns2 = Dense(200)(final_dns1)
        final_dns3 = Dense(1)(final_dns2)                                      # (-1,200)             --- (-1,1)

        batch_size = 32
        nb_train_samples = 7620
        gp_hypers = {'lik': -2.0, 'cov':np.random.rand(200,1)}

        gp = GP(gp_hypers,
            batch_size=batch_size,
            nb_train_samples=nb_train_samples)
        outputs = [gp(final_dns3)]
        model = Model(inputs=[train_dist, train_met_aq, test_met], outputs=outputs)

        # Compile the model
        loss = [gen_gp_loss(gp) for gp in model.layers]
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-2), loss=loss)
        return model

### Model train and test

In [ ]:
np.random.seed(0)
set_seed(0)
path = '../../../data_and_results/u-air/production/pm25_beijing_best36/quadratic/'
n_folds = 6

init = time()
fold_history = []
for fold in [str(i) for i in range(n_folds)]:
    clear_output(wait=True)
    print('Training fold',fold)
    # Load data
    train_dst = np.load(path+'data/fold_'+fold+'/train/adain/trn_dst.npy', allow_pickle=True).astype(np.float32)
    train_met_aq = np.load(path+'data/fold_'+fold+'/train/adain/trn_metaq.npy').astype(np.float32)
    test_met = np.load(path+'data/fold_'+fold+'/train/adain/tst_met.npy').astype(np.float32)
    test_aq = np.load(path+'data/fold_'+fold+'/train/adain/tst_aqi.npy').astype(np.float32)
    yscaler = pd.read_pickle(path+'data/fold_'+fold+'/scaler/adain/yscaler.pickle')
    print(train_dst.shape)
    # Define and train the model
#     es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
    model = ADAIN(time_window=24, met_dim=4)
    history = model.fit(X=[train_dst, train_met_aq, test_met], Y=test_aq, 
          batch_size=32, epochs=10, validation_split=0.1, verbose=2)#, callbacks=[es])
    fold_history.append(history)
    
    # Test the model
    train_dst = np.load(path+'data/fold_'+fold+'/test/adain/trn_dst.npy', allow_pickle=True).astype(np.float32)
    train_met_aq = np.load(path+'data/fold_'+fold+'/test/adain/trn_metaq.npy').astype(np.float32)
    test_met = np.load(path+'data/fold_'+fold+'/test/adain/tst_met.npy').astype(np.float32)
    test_aq = np.load(path+'data/fold_'+fold+'/test/adain/tst_aqi.npy').astype(np.float32)
    
    pred_y = np.nan*np.zeros((train_dst.shape[0], train_dst.shape[-1]))
    for test_id in range(train_dst.shape[-1]):
        pred_yy = model.predict(x=[train_dst[:,:,:,test_id], train_met_aq, test_met[:,:,:,test_id]])
        pred_y[:,test_id] = yscaler.inverse_transform(pred_yy).ravel()
    
    if not os.path.exists(path+'results/adain/fold_'+fold):
        os.makedirs(path+'results/adain/fold_'+fold)
    np.save(path+'results/adain/fold_'+fold+'/pred_y.npy', pred_y)
    np.save(path+'results/adain/fold_'+fold+'/test_y.npy', test_aq)

print('Finished in',round(time()-init)/60,'minutes')

Training fold 0
(7620, 2, 29)


In [ ]:
path = '../../../data_and_results/u-air/production/pm25_beijing_best36/quadratic/'
folds = [str(i) for i in range(6)]

### Plotting training curves

In [ ]:
fig, ax = plt.subplots(2, len(folds)//2, figsize=(15,6))
ax = ax.ravel()
for i in range(len(folds)):
    ax[i].plot(fold_history[i].history['loss'], label='train loss')
    ax[i].plot(fold_history[i].history['val_loss'], label='validation loss')
plt.legend();

### Checking results

In [ ]:
models = ['svr', 'gp_rbf', 'gp_m32', 'gp_m12', 'gp_linear','nsgp_rbf','adain']
res = pd.DataFrame(index=models, columns=['fold_'+str(i) for i in range(len(folds))]+['avg'])
# for model in models[:-1]:
#     pred_y, test_y = load_results(path, folds, f_ids, n_test, model)
#     fold_rmse = fold_wise_rmse(pred_y, test_y, len(folds))
#     fold_rmse.append(np.mean(fold_rmse))
#     res.loc[model, :] = fold_rmse

# ADAIN
for fold in folds:
    pred_y = np.load(path+'results/adain/fold_'+fold+'/pred_y.npy')
    test_y = np.load(path+'results/adain/fold_'+fold+'/test_y.npy')
    res.loc['adain', 'fold_'+fold] = mean_squared_error(test_y.ravel(), pred_y.ravel(), squared=False)
res.loc['adain','avg'] = res.loc['adain'].mean()

res

## Plotting predictions on test

In [ ]:
plt.plot(test_aq.ravel(), test_aq.ravel())
plt.scatter(test_aq.ravel(), pred_y.ravel());
plt.xlabel('PM2.5 (Ground truth)');plt.ylabel('PM2.5 (Predictions)');

In [ ]:
# plt.plot(o_tests, o_tests)
plt.plot(test_aq[:,0], label='ground truth');
plt.plot(pred_y[:,0], label='predictions');
plt.xlabel('Time-stamps'); plt.ylabel('PM2.5');
plt.legend(bbox_to_anchor=(1.5,1));

In [3]:
from keras.layers import Input, SimpleRNN
from keras.optimizers import Adam

from kgp.layers import GP
from kgp.models import Model
from kgp.losses import gen_gp_loss

input_shape = (10, 2)  # 10 time steps, 2 dimensions
batch_size = 32
nb_train_samples = 512
gp_hypers = {'lik': -2.0, 'cov': [[-0.7], [0.0]]}

# Build the model
inputs = Input(shape=input_shape)
rnn = SimpleRNN(32)(inputs)
gp = GP(gp_hypers,
        batch_size=batch_size,
        nb_train_samples=nb_train_samples)
outputs = [gp(rnn)]
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
loss = [gen_gp_loss(gp) for gp in model.output_layers]
model.compile(optimizer=Adam(1e-2), loss=loss)

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'